Delta_Live_Tables

In [0]:
import dlt
from pyspark import *
from pyspark.sql import functions 
from pyspark.sql.functions import *



#Customer_Segmentation
@dlt.table
def customer_segmentation():
    df=spark.read.table("blinkit.silver.customers")
    df=df.groupBy("customer_segment") \
             .agg(
                 functions.count("customer_id").alias("num_customers"),
                 functions.round(functions.avg("total_orders"),2).alias("avg_total_orders"),
                 functions.round(functions.avg("avg_order_value"), 2).alias("avg_order_value"),
                 functions.sum("total_orders").alias("total_orders_segment"),
                 functions.round(functions.sum((functions.col("avg_order_value") * functions.col("total_orders"))),2).alias("total_revenue")
             )
    return df



#Customer_Region_Summary
@dlt.table
def customer_region_summary():
    df=spark.read.table("blinkit.silver.customers").groupBy("area").count().withColumnRenamed("count", "num_of_customers")
    return df



#Products_MarginsPercentages
@dlt.table
def products_marginspercentages():
    df = spark.read.table("blinkit.silver.products")
    df=df.select("product_name", "margin_percentage") \
        .withColumn("margin_classification",when(col("margin_percentage") >= 25, "High").otherwise("Low")) \
        .select("product_name", "margin_percentage", "margin_classification")
    return df



#Monthly_Sales_Summary
@dlt.table
def monthly_sales_summary():
    orders_df = spark.read.table("blinkit.silver.orders")

    order_items_df = spark.read.table("blinkit.silver.orderitems")

    joined_df = orders_df.join(order_items_df, "order_id", "inner")

    monthly_orders_df = joined_df.withColumn("order_month", month(col("order_date"))) \
                             .withColumn("order_year", year(col("order_date"))) \
                             .withColumn("item_sales", col("quantity") * col("unit_price"))

    
    monthly_summary_df = monthly_orders_df.groupBy("order_year", "order_month") \
        .agg(
            countDistinct("order_id").alias("total_orders"),
            sum("quantity").alias("total_quantity_sold"),
            round(sum("item_sales"),2).alias("total_sales")
        ) \
        .orderBy("order_year", "order_month")

    return monthly_summary_df



#Delivery_Performace_Summary
@dlt.table
def delivery_performance_summary():
    delivery_df = spark.read.table("blinkit.silver.deliveryperformance")

  
    delivery_df=delivery_df.groupBy("delivery_status") \
        .agg(count("order_id").alias("number_of_orders")) \
        .select("delivery_status", "number_of_orders") \
        .orderBy(col("number_of_orders").desc())
    
    return delivery_df



#
@dlt.table
def feedback_summary():
    feedback_df = spark.read.table("blinkit.silver.customerfeedback")
    feedback_df = feedback_df.groupBy("feedback_category", "sentiment") \
        .agg(count("*").alias("count")) \
        .orderBy("feedback_category", col("count").desc())
    
    return feedback_df
